# Interest Rate Swaps Fundamentals

This notebook demonstrates how to access and use the functionalities of **Swaps** which are part of our **QPS** module within LSEG Financial Analytics SDK.

In [1]:
from lseg_analytics.pricing.instruments.ir_swaps import search, load, delete

Search code:

In [2]:
# execute the search of IR swaps
available_swaps = search()

print(available_swaps)

[{'type': 'IrSwap', 'id': '186e9289-8b77-4e95-8451-534dd439f8bf', 'location': {'space': 'test', 'name': 'SwapResourceCreatdforTest'}, 'description': {'summary': '', 'tags': ['test']}}]


### Loading a specific swap template

User can load a specific swap template by either providing a unique template name or resource ID.

Mandatory arguments: 
- resource_id - mutually exclusive with name
- name - mutually exclusive with resource_id

In [3]:
# fetch all available swaps
available_swaps = search()

# execute the load of a swap using the first element of previously fetched data
loaded_swap = load(resource_id=available_swaps[0].id)

print(loaded_swap)

<IrSwap space='test' name='SwapResourceCreatdforTest' 186e9289‥>


### Deleting a specific swap template

In [4]:
# fetch all available swaps
available_swaps = search()

# execute the load of a swap using the first element of previously fetched data
loaded_swap = load(resource_id=available_swaps[0].id)

# Clone swap to save original
cloned_swap = loaded_swap.clone()

# Save the cloned swap to a space
cloned_swap.save(name='swap_for_deletion', space='HOME')

# Delete cloned swap
delete(resource_id=cloned_swap.id)


True

## Valuing interest rate swaps

### Imports

Import the following necessary modules:

In [5]:
from lseg_analytics.pricing.instruments.ir_swaps import IrSwapDefinitionInstrument, IrPricingParameters, IrSwapSolvingParameters, IrSwapSolvingVariable, IrSwapSolvingTarget, IrMeasure, IrPricingParameters
from lseg_analytics.pricing.instruments.ir_swaps import solve, value, create_from_vanilla_irs_template, create_from_cbs_template, create_from_ccs_template, create_from_leg_template, create_from_tbs_template

import json as js

### Template-based setup for swap valuation

Using an existing swap template to build a swap, user can adjust it with his own settings.
- Swap conventions provide a standardized set of market-accepted terms, such as leg frequencies and interest calculation methods.
- By referencing a swap convention, you ensure alignment with market standards while streamlining the setup process.

In [6]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

fwd_start_sofr_def = IrSwapDefinitionInstrument(definition = fwd_start_sofr.definition)

print(js.dumps(fwd_start_sofr_def.as_dict(), indent=4))

{
    "definition": {
        "firstLeg": {
            "rate": {
                "interestRateType": "FixedRate",
                "rate": {
                    "value": 0.0,
                    "unit": "Percentage"
                }
            },
            "interestPeriods": {
                "startDate": {
                    "dateType": "RelativeAdjustableDate",
                    "tenor": "0D",
                    "referenceDate": "SpotDate"
                },
                "endDate": {
                    "dateType": "RelativeAdjustableDate",
                    "tenor": "10Y",
                    "referenceDate": "StartDate"
                },
                "frequency": "Annual",
                "businessDayAdjustment": {
                    "calendars": [
                        "USA"
                    ],
                    "convention": "NextBusinessDay"
                },
                "rollConvention": "Same"
            },
            "paymentOffset": {
        

### Solve a SOFR swap par rate

Solving a swap for a prefered value (e.g. fixed rate or spread) can be done in 2 ways:
1. Invoking a general ir_swaps method 'solve' which then requires as an input argument a swap definition object (IRSwapDefinitionInstrument)
2. Invoking a 'solve' method directly on the IRSwap Object

Both ways require preparation of execution parameters in forms of solving variable and solving target(s), as displayed in following codes.

#### General solve method

In [7]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

# prepare the Definition Instrument
fwd_start_sofr_def = IrSwapDefinitionInstrument(definition = fwd_start_sofr.definition)

# set a solving variable between first and second leg and Fixed Rate or Spread
solving_variable = IrSwapSolvingVariable(leg='FirstLeg', name='FixedRate')

# Apply solving target(s)
solving_target=IrSwapSolvingTarget(market_value=IrMeasure(value=0.0))

# Setup the solving parameter object
solving_parameters = IrSwapSolvingParameters(variable=solving_variable, target=solving_target) 

# instantiate pricing parameters
pricing_parameters = IrPricingParameters(solving_parameters=solving_parameters)

# solve the swap par rate
solving_response_general = solve(
    definitions=[fwd_start_sofr_def],
    pricing_preferences=pricing_parameters
    )

print(js.dumps(solving_response_general.as_dict(), indent=4))

{
    "pricingPreferences": {
        "valuationDate": "2025-12-03",
        "reportCurrency": "USD"
    },
    "analytics": [
        {
            "solving": {
                "result": 3.6484959229114566
            },
            "description": {
                "instrumentTag": "",
                "instrumentDescription": "Pay USD Annual 3.65% vs Receive USD Annual +0bp SOFR 2035-12-05",
                "startDate": "2025-12-05",
                "endDate": "2035-12-05",
                "tenor": "10Y"
            },
            "valuation": {
                "accrued": {
                    "value": 0.0,
                    "percent": 0.0,
                    "dealCurrency": {
                        "value": 0.0,
                        "currency": "USD"
                    },
                    "reportCurrency": {
                        "value": 0.0,
                        "currency": "USD"
                    }
                },
                "marketValue": {
             

#### Class solve method

In this case there are a few rules to abide.
1. Swap needs to be saved before solving
2. Each swap needs a unique name
3. Pricing Parameters needs to be instantiated, even if not populated explicitly

In [8]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

# Swap needs to be saved in order for the solve class method to be executable
fwd_start_sofr.save(name="sofr_fwd_start_swap_exm")

# set a solving variable between first and second leg and Fixed Rate or Spread
solving_variable = IrSwapSolvingVariable(leg='FirstLeg', name='FixedRate')

# Apply solving target(s)
solving_target=IrSwapSolvingTarget(market_value=IrMeasure(value=0.0))

# Setup the solving parameter object
solving_parameters = IrSwapSolvingParameters(variable=solving_variable, target=solving_target) 

# instantiate pricing parameters
pricing_parameters = IrPricingParameters(solving_parameters=solving_parameters)

# solve the swap par rate
solving_response_object = fwd_start_sofr.solve(pricing_preferences=pricing_parameters)

delete(name="sofr_fwd_start_swap_exm")

print(js.dumps(solving_response_object.analytics.as_dict(), indent=4))

{
    "solving": {
        "result": 3.6484959229114566
    },
    "description": {
        "instrumentTag": "",
        "instrumentDescription": "Pay USD Annual 3.65% vs Receive USD Annual +0bp SOFR 2035-12-05",
        "startDate": "2025-12-05",
        "endDate": "2035-12-05",
        "tenor": "10Y"
    },
    "valuation": {
        "accrued": {
            "value": 0.0,
            "percent": 0.0,
            "dealCurrency": {
                "value": 0.0,
                "currency": "USD"
            },
            "reportCurrency": {
                "value": 0.0,
                "currency": "USD"
            }
        },
        "marketValue": {
            "value": -4.65661287307739e-10,
            "dealCurrency": {
                "value": -4.65661287307739e-10,
                "currency": "USD"
            },
            "reportCurrency": {
                "value": -4.65661287307739e-10,
                "currency": "USD"
            }
        },
        "cleanMarketValue": {


### Value a swap

Valuing a swap can be done in 2 ways:
1. Invoking a general ir_swaps method 'value' which then requires as an input argument a swap definition object (IRSwapDefinitionInstrument)
2. Invoking a 'solve' method directly on the IRSwap Object

Both ways require preparation of execution parameters in forms of solving variable and solving target(s), as displayed in following codes.

#### General Value method

In [9]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

fwd_start_sofr_def = IrSwapDefinitionInstrument(definition = fwd_start_sofr.definition)

# instantiate pricing parameters
pricing_parameters = IrPricingParameters()

# value the swap
valuation_response = value(
    definitions=[fwd_start_sofr_def],
    pricing_preferences=pricing_parameters
)

print(js.dumps(valuation_response.analytics[0].valuation.as_dict(), indent=4))

{
    "accrued": {
        "value": 0.0,
        "percent": 0.0,
        "dealCurrency": {
            "value": 0.0,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 0.0,
            "currency": "USD"
        }
    },
    "marketValue": {
        "value": 3072227.3918728,
        "dealCurrency": {
            "value": 3072227.3918728,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 3072227.3918728,
            "currency": "USD"
        }
    },
    "cleanMarketValue": {
        "value": 3072227.3918728,
        "dealCurrency": {
            "value": 3072227.3918728,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 3072227.3918728,
            "currency": "USD"
        }
    }
}


#### Class Value method

In [10]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

# Swap needs to be saved in order for the value class method to be executable
fwd_start_sofr.save(name="sofr_fwd_start_swap_exm")

# instantiate pricing parameters
pricing_parameters = IrPricingParameters()

# solve the swap par rate
valuing_response_object = fwd_start_sofr.value(pricing_preferences=pricing_parameters)

delete(name="sofr_fwd_start_swap_exm")

print(js.dumps(valuation_response.analytics[0].valuation.as_dict(), indent=4))

{
    "accrued": {
        "value": 0.0,
        "percent": 0.0,
        "dealCurrency": {
            "value": 0.0,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 0.0,
            "currency": "USD"
        }
    },
    "marketValue": {
        "value": 3072227.3918728,
        "dealCurrency": {
            "value": 3072227.3918728,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 3072227.3918728,
            "currency": "USD"
        }
    },
    "cleanMarketValue": {
        "value": 3072227.3918728,
        "dealCurrency": {
            "value": 3072227.3918728,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 3072227.3918728,
            "currency": "USD"
        }
    }
}


#### Cashflows Output

CashFlows output is seen on the leg level, for the interest rate swap used, we can see that we have two different interest rate types of a cashflow : FixedRate (for the first swap leg) and FloatingRate (for the second swap leg)

##### 1- FixedRate CashFlows

In [11]:
fixedRate_CashFlows = [cashflow.as_dict() for cashflow in valuation_response.analytics[0].first_leg['cashflows']]
print(js.dumps(fixedRate_CashFlows, indent=4))

[
    {
        "paymentType": "Interest",
        "annualRate": {
            "value": 0.0,
            "unit": "Percentage"
        },
        "discountFactor": 0.9653209005167042,
        "startDate": "2025-12-05",
        "endDate": "2026-12-07",
        "remainingNotional": 10000000.0,
        "interestRateType": "FixedRate",
        "zeroRate": {
            "value": 3.5430972096603286,
            "unit": "Percentage"
        },
        "date": {
            "dateType": "AdjustableDate",
            "date": "2026-12-08"
        },
        "amount": {
            "value": -0.0,
            "currency": "USD"
        },
        "payer": "Party1",
        "receiver": "Party2",
        "occurrence": "Future"
    },
    {
        "paymentType": "Interest",
        "annualRate": {
            "value": 0.0,
            "unit": "Percentage"
        },
        "discountFactor": 0.9362460400267851,
        "startDate": "2026-12-07",
        "endDate": "2027-12-06",
        "remainingNotion

##### 2- FloatingRate CashFlows

For the floating rate leg, we see an indexFixings property in the cashflow data.
This section contains information about the reference rate used to determine each coupon payment — for example, SOFR, LIBOR, or EURIBOR.

In contrast, the fixed leg has no indexFixings field since its rate remains constant over the entire life of the swap.

In [12]:
floatingRate_CashFlows = [cashflow.as_dict() for cashflow in valuation_response.analytics[0].second_leg['cashflows']]
print(js.dumps(floatingRate_CashFlows, indent=4))

[
    {
        "paymentType": "Interest",
        "annualRate": {
            "value": 3.492202039114,
            "unit": "Percentage"
        },
        "discountFactor": 0.9653209005167042,
        "startDate": "2025-12-05",
        "endDate": "2026-12-07",
        "remainingNotional": 10000000.0,
        "interestRateType": "FloatingRate",
        "zeroRate": {
            "value": 3.5430972096603286,
            "unit": "Percentage"
        },
        "indexFixings": [
            {
                "accrualEndDate": "2026-12-07",
                "accrualStartDate": "2025-12-05",
                "couponRate": {
                    "value": 3.492202,
                    "unit": "Percentage"
                },
                "fixingDate": "2025-12-05",
                "forwardSource": "ZcCurve",
                "referenceRate": {
                    "value": 3.492202,
                    "unit": "Percentage"
                },
                "spreadBp": 0.0
            }
        ]

## Creating a swap from CBS template

In [13]:
swap_from_cbs = create_from_cbs_template(template_reference = "LSEG/GBUSSOSRBS")
print(swap_from_cbs.definition)

{'firstLeg': {'rate': {'interestRateType': 'FloatingRate', 'index': 'LSEG/GBP_SONIA_ON_BOE', 'spreadSchedule': [{'rate': {'value': 0.0, 'unit': 'BasisPoint'}}], 'resetDates': {'offset': {'tenor': '0D', 'businessDayAdjustment': {'calendars': [], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Backward'}}, 'leverage': 1.0}, 'interestPeriods': {'startDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '0D', 'referenceDate': 'SpotDate'}, 'endDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '10Y', 'referenceDate': 'StartDate'}, 'frequency': 'Quarterly', 'businessDayAdjustment': {'calendars': ['UKG', 'USA'], 'convention': 'ModifiedFollowing'}, 'rollConvention': 'Same'}, 'paymentOffset': {'tenor': '2D', 'businessDayAdjustment': {'calendars': ['UKG', 'USA'], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Forward'}, 'couponDayCount': 'Dcb_Actual_365', 'accrualDayCount': 'Dcb_Actual_365', 'principal': {'currency': 

## Create a swap from CCS Template

In [14]:
swap_from_ccs = create_from_ccs_template(template_reference = "LSEG/CNUSQMSRBS")
print(swap_from_ccs.definition)

{'firstLeg': {'rate': {'interestRateType': 'FixedRate', 'rate': {'value': 0.0, 'unit': 'Percentage'}}, 'interestPeriods': {'startDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '0D', 'referenceDate': 'SpotDate'}, 'endDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '10Y', 'referenceDate': 'StartDate'}, 'frequency': 'Quarterly', 'businessDayAdjustment': {'calendars': ['CHN', 'USA'], 'convention': 'ModifiedFollowing'}, 'rollConvention': 'Same'}, 'paymentOffset': {'tenor': '2D', 'businessDayAdjustment': {'calendars': ['CHN', 'USA'], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Forward'}, 'couponDayCount': 'Dcb_Actual_360', 'accrualDayCount': 'Dcb_Actual_360', 'principal': {'currency': 'CNY', 'amount': 10000000.0, 'initialPrincipalExchange': False, 'finalPrincipalExchange': True, 'interimPrincipalExchange': False, 'repaymentCurrency': 'CNY'}, 'settlement': {'currency': 'USD'}, 'payer': 'Party1', 'receiver': 'Party2'}, 'secondLeg': {'rate'

## Create a swap from LEG Template

In [15]:
swap_from_leg = create_from_leg_template(first_leg_reference = "LSEG/EUR_AB3E_FLT", second_leg_reference = "LSEG/EUR_AB3E_FXD")
print(swap_from_leg.definition)

{'firstLeg': {'rate': {'interestRateType': 'FloatingRate', 'index': 'LSEG/EUR_EURIBOR_3M_EMMI', 'spreadSchedule': [{'rate': {'value': 0.0, 'unit': 'BasisPoint'}}], 'resetDates': {'offset': {'tenor': '2D', 'businessDayAdjustment': {'calendars': [], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodStartDate', 'direction': 'Backward'}}, 'leverage': 1.0}, 'interestPeriods': {'startDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '0D', 'referenceDate': 'SpotDate'}, 'endDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '10Y', 'referenceDate': 'StartDate'}, 'frequency': 'Quarterly', 'businessDayAdjustment': {'calendars': ['EMU'], 'convention': 'ModifiedFollowing'}, 'rollConvention': 'Same'}, 'paymentOffset': {'tenor': '0D', 'businessDayAdjustment': {'calendars': ['EMU'], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Forward'}, 'couponDayCount': 'Dcb_Actual_360', 'accrualDayCount': 'Dcb_Actual_360', 'principal': {'currency': 'EUR', 'a

## Create a swap from TBS Template

In [16]:
swap_from_tbs = create_from_tbs_template(template_reference = "LSEG/CBS_USDSR3LIMM")
print(swap_from_tbs.definition)

{'firstLeg': {'rate': {'interestRateType': 'FloatingRate', 'index': 'LSEG/USD_SOFR_ON_FRBNY', 'spreadSchedule': [{'rate': {'value': 0.0, 'unit': 'BasisPoint'}}], 'resetDates': {'offset': {'tenor': '0D', 'businessDayAdjustment': {'calendars': [], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Backward'}}, 'leverage': 1.0}, 'interestPeriods': {'startDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '0D', 'referenceDate': 'SpotDate'}, 'endDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '10Y', 'referenceDate': 'StartDate'}, 'frequency': 'Quarterly', 'businessDayAdjustment': {'calendars': ['USA'], 'convention': 'ModifiedFollowing'}, 'rollConvention': 'Same'}, 'paymentOffset': {'tenor': '2D', 'businessDayAdjustment': {'calendars': ['USA'], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Forward'}, 'couponDayCount': 'Dcb_Actual_360', 'accrualDayCount': 'Dcb_Actual_360', 'principal': {'currency': 'USD', 'amoun

### Saving a swap

In [17]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

swap_id = "SOFR_OIS_1Y2Y"

swap_space = "HOME"

try:
    # If the instrument does not exist in HOME space, we can save it
    fwd_start_sofr.save(name=swap_id, space=swap_space)
    print(f"Instrument {swap_id} saved in {swap_space} space.")
except:
    # Check if the instrument already exists in HOME space
    fwd_start_sofr = load(name=swap_id, space=swap_space)
    print(f"Instrument {swap_id} already exists in {swap_space} space.")

Instrument SOFR_OIS_1Y2Y saved in HOME space.


### Deleting the saved swap

In [18]:
# Let's delete the instrument we created in HOME space
from lseg_analytics.pricing.instruments.ir_swaps import delete

swap_id = "SOFR_OIS_1Y2Y"

delete(name=swap_id, space="HOME")

True